<a href="https://colab.research.google.com/github/junaid537/pytorch/blob/main/%E2%80%9CHW2_Junaid_Khalidi_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
url = "https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


<ipython-input-3-dd748172f966>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep='\t', compression='gzip', usecols=['review_body', 'star_rating'],on_bad_lines='skip')


In [4]:
#Ensure that star_rating contains only valid numeric values:

# Convert star_rating to numeric, coercing errors to NaN
df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')

# Drop rows with invalid star_rating values
df = df.dropna(subset=['star_rating'])

# Convert star_rating to integer (if required)
df['star_rating'] = df['star_rating'].astype(int)

<ipython-input-4-251f533e7f5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['star_rating'] = df['star_rating'].astype(int)


In [5]:
df

,star_rating,review_body
0,5,Great product.
1,5,"What's to say about this commodity item except, what have we come to in this world.<br />Having the need to bnuy captured and compressed air. &#60;lol&#62;"
2,5,"Haven't used yet, but I am sure I will like it."
3,1,"Although this was labeled as &#34;new&#34; the one I received clearly had been used. The box had previously been opened., and the shredder was dirty and the bin was partially full of shredded paper. What was worse is that the unit will not work properly. It is not possible to insert the paper bin so as to enable the shredder to run. It will not operate if the bin is not in place, but I could never get the unit to recognize that the paper bin was actually fully inserted. After cleaning everything thoroughly and vacuuming the paper bin area, it worked ONCE! After that I was unable to get it work at all. I returned the unit immediately for a refund. I feel Amazon misrepresented the unit as &#34;new&#34; when clearly it was not."
4,4,Gorgeous colors and easy to use
...,...,...
2640347,4,"I can't live anymore whithout my Palm III. But I also hate to carry things around. Enter the Belt Clip Case for PalmPilot. It is extremely light, holds thightly to my belt or jeans pocket and it features a hard plastic screen protection sewed inside. I let my Palm III (inside this case, fortunately) slip and it fell all the way down on a 17 steps stairs. Not a single scratch! I wish it only had an internal pocket for credit and business cards."
2640348,4,"Although the Palm Pilot is thin and compact it may exceed your pocket capacity. With the belt clip you can carry the Palm Pilot, enter and review data at any time. This especially useful for physicians who are on the move constantly. I have found this carrying case to be an excellent accessory."
2640349,4,This book had a lot of great content without being boring or monotonous. I really enjoy Greek mythology and this book summed it up throughly.
2640350,5,"I am teaching a course in Excel and am using the step-by-step books. They are very good. The class is composed of people who are proficient in 1-2-3, and are computer literate, so we're off and running!"


In [6]:
balanced_df = df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)
balanced_df

<ipython-input-6-7afa5d846570>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = df.groupby('star_rating').apply(lambda x: x.sample(n=50000, random_state=42)).reset_index(drop=True)


,star_rating,review_body
0,1,"This item was absolutely terrible. The ink barely showed out of my brother printer. I wanted to return it, but it was too late to return. Never buying again..."
1,1,out from the box there is a blank area when it print.
2,1,"I read ALL the reviews Bad & Good and made my Choice, Wrong Choice!!! I really, Really Love Amazon.com But they really need to Use some of these Catridges for Theirself and Evaluate Them. my Printer was Printing Fine it just needed Ink so I looked these up on Amazon and took a Chance even with more than a few Negative Reviews. I sent some Samples of the Horrible attempt at printing in the Box With The easy return for Refund. Went to WalMart and picked up 2 combo packs of Kodak ink for my Two Printers and Sure Enough it went back to Printing Perfect, Both of the Printers. Well I guess the Fact that Kodak's Ink is so Cheap that It has a tendantcy to push you to see just how cheap you can get it for. Please Folks Save your Time and Frustrations for more Important Troubles in your life, Don't Be Greedy. Buy the Kodak Ink you Won't be Dissapointed. Thanks Amazon, for Your easy Returns, because with out them I would really be up the Creek without a Paddle. I just can't say enough, \\""How much I love Amazon.com\\"" Thanks,"
3,1,This came very sharp and it would not come apart. I would not recommend buying this because it cut my finger and it came made wrong. This broke in one day and I regret buying it. Waste of money. :(
4,1,"I'd give this day timer zero stars if that was an option. This calendar is 2 years old! It does say 2011 in the fine print - but being the sucker buyer I did not realize I'd have to verify if it was current - and I paid almost $20 for this old, useless calendar."
...,...,...
249995,5,LO QUE BUSCABA PARA MI OFICINA EN CASA MUY BUEN PRODUCTO LO RECOMIENDO PARA PERSONAS QUE POSEEN SUS OFICINAS A
249996,5,"I have had Panasonic 3 phones for a long time. Recently I had no phones, internet, or security system. I decided to start with new phones. After reading all of the reviews I decided this was the phone system I needed. They came very fast. Still no phone. The phone company came out and found a broken cable leading into my house. Once this was fixed I instantly had phones, internet, & security system. I wish I had ordered a four phone system as I found another area I would like a phone. So, checked with Amazon and instantly found one. It arrived before scheduled delivery date. I love the \\""talking\\"" feature. I do not bother to answer my phone when it tells me \\""out of area\\"". I am sure my older phones still work!* Love Panasonic & Amazon!."
249997,5,These are great pens to draw with and I like the softness of the brush side and the fine tip on the other.
249998,5,"I do martial arts, Jiu-Jitsu and I like to take notes and this book is sturdy and it can take a beating in the room when I am throwing it around after the instructional to the side before drilling. It is great for taking notes in general and good solid book. You can definitely defend yourself with this thing. If this book can handle my roughness it can handle your basic needs, will buy again."


In [7]:
def map_sentiment(rating):
    if rating > 3:
        return 1  # Positive
    elif rating < 3:
        return 2  # Negative
    else:
        return 3  # Neutral

balanced_df['sentiment'] = balanced_df['star_rating'].apply(map_sentiment)
orig_df = balanced_df.copy()
balanced_df

,star_rating,review_body,sentiment
0,1,"This item was absolutely terrible. The ink barely showed out of my brother printer. I wanted to return it, but it was too late to return. Never buying again...",2
1,1,out from the box there is a blank area when it print.,2
2,1,"I read ALL the reviews Bad & Good and made my Choice, Wrong Choice!!! I really, Really Love Amazon.com But they really need to Use some of these Catridges for Theirself and Evaluate Them. my Printer was Printing Fine it just needed Ink so I looked these up on Amazon and took a Chance even with more than a few Negative Reviews. I sent some Samples of the Horrible attempt at printing in the Box With The easy return for Refund. Went to WalMart and picked up 2 combo packs of Kodak ink for my Two Printers and Sure Enough it went back to Printing Perfect, Both of the Printers. Well I guess the Fact that Kodak's Ink is so Cheap that It has a tendantcy to push you to see just how cheap you can get it for. Please Folks Save your Time and Frustrations for more Important Troubles in your life, Don't Be Greedy. Buy the Kodak Ink you Won't be Dissapointed. Thanks Amazon, for Your easy Returns, because with out them I would really be up the Creek without a Paddle. I just can't say enough, \\""How much I love Amazon.com\\"" Thanks,",2
3,1,This came very sharp and it would not come apart. I would not recommend buying this because it cut my finger and it came made wrong. This broke in one day and I regret buying it. Waste of money. :(,2
4,1,"I'd give this day timer zero stars if that was an option. This calendar is 2 years old! It does say 2011 in the fine print - but being the sucker buyer I did not realize I'd have to verify if it was current - and I paid almost $20 for this old, useless calendar.",2
...,...,...,...
249995,5,LO QUE BUSCABA PARA MI OFICINA EN CASA MUY BUEN PRODUCTO LO RECOMIENDO PARA PERSONAS QUE POSEEN SUS OFICINAS A,1
249996,5,"I have had Panasonic 3 phones for a long time. Recently I had no phones, internet, or security system. I decided to start with new phones. After reading all of the reviews I decided this was the phone system I needed. They came very fast. Still no phone. The phone company came out and found a broken cable leading into my house. Once this was fixed I instantly had phones, internet, & security system. I wish I had ordered a four phone system as I found another area I would like a phone. So, checked with Amazon and instantly found one. It arrived before scheduled delivery date. I love the \\""talking\\"" feature. I do not bother to answer my phone when it tells me \\""out of area\\"". I am sure my older phones still work!* Love Panasonic & Amazon!.",1
249997,5,These are great pens to draw with and I like the softness of the brush side and the fine tip on the other.,1
249998,5,"I do martial arts, Jiu-Jitsu and I like to take notes and this book is sturdy and it can take a beating in the room when I am throwing it around after the instructional to the side before drilling. It is great for taking notes in general and good solid book. You can definitely defend yourself with this thing. If this book can handle my roughness it can handle your basic needs, will buy again.",1


In [8]:
#train_df, test_df = train_test_split(balanced_df, test_size=0.2, stratify=balanced_df['sentiment'], random_state=42)
print(orig_df)

        star_rating  \
0                 1   
1                 1   
2                 1   
3                 1   
4                 1   
...             ...   
249995            5   
249996            5   
249997            5   
249998            5   
249999            5   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [9]:
pip install gensim

In [10]:
import pandas as pd
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import word_tokenize
import numpy as np
import nltk

# Download NLTK tokenizer
nltk.download('punkt')
import gensim.downloader as api
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk

# Download tokenizer
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
# Load pretrained Word2Vec model (Google News 300)
wv = api.load("word2vec-google-news-300")

# Check the vector representation of a word
#print(wv["excellent"])  # A 300-dimensional vector

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [12]:
#check semantic similarities
result = wv.most_similar(positive=["king", "woman"], negative=["man"], topn=1)
print("King - Man + Woman =", result[0][0])  # Expected output: 'queen'

King - Man + Woman = queen


In [13]:
#similarities between words
similarity = wv.similarity("excellent", "outstanding")
print("Similarity between 'excellent' and 'outstanding':", similarity)

Similarity between 'excellent' and 'outstanding': 0.55674857


In [14]:
import nltk
nltk.download('punkt_tab')
#tokenize your dataset
# Tokenize the review text
balanced_df["tokens"] = balanced_df["review_body"].astype(str).apply(word_tokenize)
# Show some tokenized reviews
print(balanced_df["tokens"].head())


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


0                                                                                                                                                                                                                                                                                                                                                                                                                [This, item, was, absolutely, terrible, ., The, ink, barely, showed, out, of, my, brother, printer, ., I, wanted, to, return, it, ,, but, it, was, too, late, to, return, ., Never, buying, again, ...]
1                                                                                                                                                                                                                                                                                                                                                                                                              

In [15]:
#converting to word embeddings
'''
def get_word2vec_embedding(tokens, model):
    vectors = [model[word] for word in tokens if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)  # 300D vector

# Applying to dataset using the pretrained Word2Vec model
balanced_df["embedding"] = balanced_df["tokens"].apply(lambda x: get_word2vec_embedding(x, wv))'''
import gensim.downloader as api
import numpy as np
import pandas as pd

# Load the pretrained Word2Vec model
wv = api.load("word2vec-google-news-300")

# Function to get embeddings for each word in a review
def get_word_embeddings(tokens, model):
    return [model[word] for word in tokens if word in model]  # List of word embeddings

# Load dataset (assume it's preprocessed and tokenized)
#df = pd.read_pickle("amazon_reviews.pkl")  # Replace with actual dataset file

# Apply function to dataset
balanced_df["word_embeddings"] = balanced_df["tokens"].apply(lambda x: get_word_embeddings(x, wv))

###(b) Training Word2Vec model on our own datset

In [16]:
# Training my custom Word2Vec model
custom_w2v = Word2Vec(
    sentences=balanced_df["tokens"],  # List of tokenized reviews
    vector_size=300,         # Word embedding size
    window=11,               # Context window size
    min_count=10,            # Minimum word frequency threshold
    workers=4                # Use multiple CPU cores for efficiency
)

# Save the trained model
custom_w2v.save("custom_word2vec.model")

# Example: Check words most similar to "excellent"
print(custom_w2v.wv.most_similar("excellent"))

[('outstanding', 0.8622373938560486), ('exceptional', 0.8050286173820496), ('terrific', 0.7273557186126709), ('superb', 0.7133903503417969), ('fantastic', 0.7097468972206116), ('amazing', 0.6862058639526367), ('incredible', 0.6634069085121155), ('wonderful', 0.6506627202033997), ('great', 0.6504523754119873), ('awesome', 0.6457569003105164)]


In [17]:
result_custom = custom_w2v.wv.most_similar(positive=["king", "woman"], negative=["man"], topn=1)
print("Custom Word2Vec: King - Man + Woman =", result_custom[0][0])

Custom Word2Vec: King - Man + Woman = 1/4


In [18]:
similarity_custom = custom_w2v.wv.similarity("excellent", "outstanding")
print("Custom Word2Vec similarity (excellent vs outstanding):", similarity_custom)

Custom Word2Vec similarity (excellent vs outstanding): 0.8622374


**Comparison based on King - Man + Woman on above 2 models :**

Google News Word2Vec : Returns "Queen"

Custom Word2Vec (Amazon Reviews) : Does not return "Queen" as the dataset may lacks historical context.


**Comparison based on Excellent vs Outstanding :**


Google News Word2Vec : High similarity (~0.9)


Custom Word2Vec (Amazon Reviews) :0.86168975 ( lower due to limited data.)


**Based on Context Understanding**

Google News Word2Vec : General knowledge	Biased towards Amazon review words.

Custom Word2Vec (Amazon Reviews): Biased towards Amazon review words.

**Conclusion:-**

Pretrained Word2Vec (Google News 300) performs better for general language tasks

It was trained on a large-scale dataset (Google News corpus).
It encodes rich semantic relationships between words.
It correctly captures King - Man + Woman = Queen.

Custom Word2Vec is more domain-specific (Amazon Reviews)

It is trained on customer reviews, so it understands sentiment-related words well.
But it lacks general world knowledge, making it weaker for analogy tasks.
Example: It may not return "Queen" for King - Man + Woman, but it might capture sentiment-related similarities better (e.g., "great" ∼ "awesome")

**Best Choice?**

If we need general word understanding, use then pretrained Word2Vec.

If we need domain-specific representations (Amazon reviews, legal, medical, etc.),  our own custom trained Word2V on reviews works better

In [19]:
print(custom_w2v.wv.most_similar("excellent"))

[('outstanding', 0.8622373938560486), ('exceptional', 0.8050286173820496), ('terrific', 0.7273557186126709), ('superb', 0.7133903503417969), ('fantastic', 0.7097468972206116), ('amazing', 0.6862058639526367), ('incredible', 0.6634069085121155), ('wonderful', 0.6506627202033997), ('great', 0.6504523754119873), ('awesome', 0.6457569003105164)]


### 3.

### Data cleaning and Data Preprocessing

In [20]:
contractions_dict = {
    "aren't": "are not",
    "can't": "cannot",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he's": "he is",
    "how's": "how is",
    "i'd": "I would",
    "i'll": "I will",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it's": "it is",
    "let's": "let us",
    "mightn't": "might not",
    "mustn't": "must not",
    "shan't": "shall not",
    "she's": "she is",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they would",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "we'd": "we would",
    "we're": "we are",
    "weren't": "were not",
    "what's": "what is",
    "where's": "where is",
    "who's": "who is",
    "won't": "will not",
    "wouldn't": "would not",
    "you'd": "you would",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have"
}

In [21]:
df=orig_df.copy()
df

,star_rating,review_body,sentiment
0,1,"This item was absolutely terrible. The ink barely showed out of my brother printer. I wanted to return it, but it was too late to return. Never buying again...",2
1,1,out from the box there is a blank area when it print.,2
2,1,"I read ALL the reviews Bad & Good and made my Choice, Wrong Choice!!! I really, Really Love Amazon.com But they really need to Use some of these Catridges for Theirself and Evaluate Them. my Printer was Printing Fine it just needed Ink so I looked these up on Amazon and took a Chance even with more than a few Negative Reviews. I sent some Samples of the Horrible attempt at printing in the Box With The easy return for Refund. Went to WalMart and picked up 2 combo packs of Kodak ink for my Two Printers and Sure Enough it went back to Printing Perfect, Both of the Printers. Well I guess the Fact that Kodak's Ink is so Cheap that It has a tendantcy to push you to see just how cheap you can get it for. Please Folks Save your Time and Frustrations for more Important Troubles in your life, Don't Be Greedy. Buy the Kodak Ink you Won't be Dissapointed. Thanks Amazon, for Your easy Returns, because with out them I would really be up the Creek without a Paddle. I just can't say enough, \\""How much I love Amazon.com\\"" Thanks,",2
3,1,This came very sharp and it would not come apart. I would not recommend buying this because it cut my finger and it came made wrong. This broke in one day and I regret buying it. Waste of money. :(,2
4,1,"I'd give this day timer zero stars if that was an option. This calendar is 2 years old! It does say 2011 in the fine print - but being the sucker buyer I did not realize I'd have to verify if it was current - and I paid almost $20 for this old, useless calendar.",2
...,...,...,...
249995,5,LO QUE BUSCABA PARA MI OFICINA EN CASA MUY BUEN PRODUCTO LO RECOMIENDO PARA PERSONAS QUE POSEEN SUS OFICINAS A,1
249996,5,"I have had Panasonic 3 phones for a long time. Recently I had no phones, internet, or security system. I decided to start with new phones. After reading all of the reviews I decided this was the phone system I needed. They came very fast. Still no phone. The phone company came out and found a broken cable leading into my house. Once this was fixed I instantly had phones, internet, & security system. I wish I had ordered a four phone system as I found another area I would like a phone. So, checked with Amazon and instantly found one. It arrived before scheduled delivery date. I love the \\""talking\\"" feature. I do not bother to answer my phone when it tells me \\""out of area\\"". I am sure my older phones still work!* Love Panasonic & Amazon!.",1
249997,5,These are great pens to draw with and I like the softness of the brush side and the fine tip on the other.,1
249998,5,"I do martial arts, Jiu-Jitsu and I like to take notes and this book is sturdy and it can take a beating in the room when I am throwing it around after the instructional to the side before drilling. It is great for taking notes in general and good solid book. You can definitely defend yourself with this thing. If this book can handle my roughness it can handle your basic needs, will buy again.",1


In [22]:
#from locale import D_FMT
import pandas as pd
import re
import nltk
from nltk.corpus import words
import re
#from spellchecker import SpellChecker
import pandas as pd
import re

# Initialize spell checker
#spell = SpellChecker()

# Function to filter misspelled words
def remove_misspelled_words(text):
    if isinstance(text, str):
        # Split text into words
        words = text.split()
        # Check each word and retain it only if it's correctly spelled
        corrected_words = [word for word in words if word in spell]
        return " ".join(corrected_words)
    return ""

# Download the English words corpus (only required once)
nltk.download('words')

# Get the set of valid English words
english_words = set(words.words())

# Function to filter non-English words from a string
def filter_non_english_words(text):
    if isinstance(text, str):
        # Split text into words, filter non-English words, and rejoin
        filtered_words = [word for word in text.split() if word.lower() in english_words]
        return " ".join(filtered_words)
    return ""

def expand_contractions(text, contractions_map):
    if isinstance(text, str):
        pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in contractions_map.keys()) + r')\b')
        return pattern.sub(lambda x: contractions_map[x.group()], text)
    return text

def preprocess_reviews(df):
    # Step 1: Convert all reviews to lowercase
    df['cleaned_review'] = df['review_body'].str.lower()

    # Step 2: Remove HTML tags and URLs
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'<.*?>', '', x) if isinstance(x, str) else '')
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'http\S+', '', x) if isinstance(x, str) else '')

    # Step 3: Expand contractions (e.g., can't -> can not)
    #df['cleaned_review'] = df['cleaned_review'].apply(lambda x: contractions.fix(x))
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: expand_contractions(x, contractions_dict))


    # Step 4: Separate concatenated words (e.g., slot...can't -> slot can't)
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'(\w)(\'\w)', r'\1 \2', x))  # Insert space between word and contraction

    # Step 5: Replace multiple dots (two or more) with a space
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'\.{2,}', ' ', x))

    # Step 6: Replace non-alphabetical characters (except for apostrophes and spaces) with a space
    #df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'[^\w\s\'-]', ' ', x))  # Keep alphabet, spaces, apostrophes, and hyphens
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'[^a-zA-Z\s\']', ' ', x))  # Keep only alphabet, spaces, and apostrophes
    # Step 7: Remove extra spaces (e.g., from replacing punctuation)
    df['cleaned_review'] = df['cleaned_review'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

    # Step 8:  Apply the function to remove misspelled words
    df['cleaned_review'] = df['cleaned_review'].apply(filter_non_english_words)
    #df['cleaned_review'] = df['cleaned_review'].apply(remove_misspelled_words)
    return df

def average_length(df, column='review_body'):
    # Replace non-string values with empty string
    df[column] = df[column].fillna('').astype(str)

    # Calculate average length
    lengths = df[column].apply(len)
    return lengths.mean()



# Calculate average lengths before preprocessing
avg_length_before = average_length(df, column='review_body')

# Preprocess the reviews
reduced_df = preprocess_reviews(df)

# Calculate average lengths after preprocessing
avg_length_after = average_length(df, column='cleaned_review')

# Output average lengths before and after
print(f"Average length before Data Cleaning: {avg_length_before}, Average length after Data Cleaning: {avg_length_after}")



#Average length before cleaning: 319.694505
#Average length after cleaning: 260.67447

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


Average length before Data Cleaning: 342.972252, Average length after Data Cleaning: 280.349656


In [23]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Download necessary resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to get the WordNet POS tag from the NLTK POS tag
def get_wordnet_pos(tag):
    if tag.startswith('VB'):  # Verbs
        return wordnet.VERB
    elif tag.startswith('JJ'):  # Adjectives
        return wordnet.ADJ
    elif tag.startswith('NN'):  # Nouns
        return wordnet.NOUN
    elif tag.startswith('RB'):  # Adverbs
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if no match

# Function to lemmatize text with POS tagging
def lemmatize_text(text):
    # Tokenize the sentence and POS tag each token
    tokens = word_tokenize(text)
    tagged_tokens = nltk.pos_tag(tokens)

    # Lemmatize using POS tags and filter out stopwords
    lemmatized_words = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        for word, tag in tagged_tokens if word.lower() not in stop_words
    ]

    # Return the lemmatized sentence as a string
    return ' '.join(lemmatized_words)

# Apply lemmatization to your dataframe column
df['cleaned_review'] = df['cleaned_review'].apply(lemmatize_text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [24]:
print(f"Average length before data preprocessing: {avg_length_after}, Average length after data preprocessing: {average_length(reduced_df, column='cleaned_review')}")


Average length before data preprocessing: 280.349656, Average length after data preprocessing: 153.755496


In [25]:
df_3classes = df.copy()
# Labels: Sentiment classification (Class 1 = Positive, Class 2 = Negative)
df = df[df["sentiment"].isin([1, 2])]  # Ignore Class 3 (Neutral)

In [26]:
df_3classes.shape

(250000, 4)

In [27]:
filtered_df = df[df['sentiment'] == 5]
print(filtered_df)

Empty DataFrame
Columns: [star_rating, review_body, sentiment, cleaned_review]
Index: []


In [28]:
### Computing Word2Vec Embeddings from Google and Custom
from gensim.models import Word2Vec
import gensim.downloader as api

# Load pretrained Word2Vec (Google News 300)
wv_google = api.load("word2vec-google-news-300")

# Load custom-trained Word2Vec
custom_w2v = Word2Vec.load("custom_word2vec.model")

# Function to compute average word embedding for a review
def get_avg_embedding(tokens, model):
    vectors = [model[word] for word in tokens.split() if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)  # Return zero vector if no valid words

# Compute embeddings for both models
df["embedding_google"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
df["embedding_custom"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))

<ipython-input-28-724c759ae486>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["embedding_google"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
<ipython-input-28-724c759ae486>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["embedding_custom"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))


In [29]:
s="uiuiu hjhj jhj"
for word in s.split():
  print(word)

uiuiu
hjhj
jhj


In [30]:
### Tf-idf feature extraction
newdf= reduced_df
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,  # Limit to top 5000 terms (optional)
    stop_words='english',  # Remove common stop words
    #ngram_range=(1,2)  # Use unigrams and bigrams
)
# Fit the vectorizer and transform the data
tfidf_features = tfidf_vectorizer.fit_transform(df['cleaned_review'])

In [31]:
tfidf_features

<200000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 3221912 stored elements in Compressed Sparse Row format>

In [32]:
# Train - Test Split
from sklearn.model_selection import train_test_split
# Train-test split (80% training, 20% testing)
X_train_google, X_test_google, y_train, y_test = train_test_split(
    list(df["embedding_google"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_custom, X_test_custom, _, _ = train_test_split(
    list(df["embedding_custom"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_tfidf, X_test_tfidf, _, _ = train_test_split(
    tfidf_features, df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

In [33]:
print(type(X_train_google))  # <class 'list'>
print(type(X_train_google[0]))  # <class 'numpy.ndarray'>
print(X_train_google[0].shape)  # (300,)

<class 'list'>
<class 'numpy.ndarray'>
(300,)


In [34]:
print(type(X_train_custom))  # <class 'list'>
print(type(X_train_custom[0]))  # <class 'numpy.ndarray'>
print(X_train_custom[0].shape)  # (300,)

<class 'list'>
<class 'numpy.ndarray'>
(300,)


In [35]:
#Perceptron training
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to evaluate model performance
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Training metrics
    train_acc = accuracy_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred, average="binary")
    train_recall = recall_score(y_train, y_train_pred, average="binary")
    train_f1 = f1_score(y_train, y_train_pred, average="binary")

    # Test metrics
    test_acc = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred, average="binary")
    test_recall = recall_score(y_test, y_test_pred, average="binary")
    test_f1 = f1_score(y_test, y_test_pred, average="binary")

    # Print results
    print(f"\n📌 {model_name} Results:")
    print(f"✅ Training Accuracy: {train_acc:.4f} | Test Accuracy: {test_acc:.4f}")
    print(f"✅ Training Precision: {train_precision:.4f} | Test Precision: {test_precision:.4f}")
    print(f"✅ Training Recall: {train_recall:.4f} | Test Recall: {test_recall:.4f}")
    print(f"✅ Training F1 Score: {train_f1:.4f} | Test F1 Score: {test_f1:.4f}")

    #return {
    #    "train_acc": train_acc, "test_acc": test_acc,
    #    "train_precision": train_precision, "test_precision": test_precision,
    #    "train_recall": train_recall, "test_recall": test_recall,
    #    "train_f1": train_f1, "test_f1": test_f1
    #}

### **1️⃣ Perceptron Models**
print("\n🔹 Training Perceptron Models...")
perc_google = Perceptron()
perc_google.fit(X_train_google, y_train)
evaluate_model(perc_google, X_train_google, X_test_google, y_train, y_test, "Perceptron (Google News W2V)")

perc_custom = Perceptron()
perc_custom.fit(X_train_custom, y_train)
evaluate_model(perc_custom, X_train_custom, X_test_custom, y_train, y_test, "Perceptron (Custom W2V)")

perc_tfidf = Perceptron()
perc_tfidf.fit(X_train_tfidf, y_train)
evaluate_model(perc_tfidf, X_train_tfidf, X_test_tfidf, y_train, y_test, "Perceptron (TF-IDF)")


### **2️⃣ SVM Models**
print("\n🔹 Training SVM Models...")
svm_google = LinearSVC(random_state=42, max_iter=10000)
svm_google.fit(X_train_google, y_train)
evaluate_model(svm_google, X_train_google, X_test_google, y_train, y_test, "SVM (Google News W2V)")

svm_custom = LinearSVC(random_state=42, max_iter=10000)
svm_custom.fit(X_train_custom, y_train)
evaluate_model(svm_custom, X_train_custom, X_test_custom, y_train, y_test, "SVM (Custom W2V)")

svm_tfidf =LinearSVC(random_state=42, max_iter=10000)
svm_tfidf.fit(X_train_tfidf, y_train)
evaluate_model(svm_tfidf, X_train_tfidf, X_test_tfidf, y_train, y_test, "SVM (TF-IDF)")



🔹 Training Perceptron Models...

📌 Perceptron (Google News W2V) Results:
✅ Training Accuracy: 0.7853 | Test Accuracy: 0.7836
✅ Training Precision: 0.7921 | Test Precision: 0.7889
✅ Training Recall: 0.7738 | Test Recall: 0.7743
✅ Training F1 Score: 0.7828 | Test F1 Score: 0.7816

📌 Perceptron (Custom W2V) Results:
✅ Training Accuracy: 0.7423 | Test Accuracy: 0.7404
✅ Training Precision: 0.6838 | Test Precision: 0.6824
✅ Training Recall: 0.9013 | Test Recall: 0.8995
✅ Training F1 Score: 0.7776 | Test F1 Score: 0.7761

📌 Perceptron (TF-IDF) Results:
✅ Training Accuracy: 0.8109 | Test Accuracy: 0.7904
✅ Training Precision: 0.8006 | Test Precision: 0.7813
✅ Training Recall: 0.8280 | Test Recall: 0.8066
✅ Training F1 Score: 0.8141 | Test F1 Score: 0.7938

🔹 Training SVM Models...

📌 SVM (Google News W2V) Results:
✅ Training Accuracy: 0.8122 | Test Accuracy: 0.8125
✅ Training Precision: 0.8314 | Test Precision: 0.8312
✅ Training Recall: 0.7833 | Test Recall: 0.7844
✅ Training F1 Score: 0.806

### TF-IDF remains the best performing feature type for both Perceptron and SVM, achieving: Best test accuracy (84.1%) with SVM.

### SVM outperforms Perceptron consistently.

### Custom Word2Vec benefits from domain-specific training but still lags behind TF-IDF.

In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import gensim.downloader as api
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [77]:
df = df_3classes.copy()
df = df[df["sentiment"].isin([1, 2])]
from sklearn.model_selection import train_test_split

### Computing Word2Vec Embeddings from Google and Custom
from gensim.models import Word2Vec
import gensim.downloader as api

# Load pretrained Word2Vec (Google News 300)
wv_google = api.load("word2vec-google-news-300")

# Load custom-trained Word2Vec
custom_w2v = Word2Vec.load("custom_word2vec.model")

# Function to compute average word embedding for a review
def get_avg_embedding(tokens, model):
    vectors = [model[word] for word in tokens.split() if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)  # Return zero vector if no valid words

# Compute embeddings for both models
df["embedding_google"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
df["embedding_custom"] = df["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))



KeyError: 'embedding_google'

In [83]:
# Train-test split (80% training, 20% testing)
X_train_google, X_test_google, y_train, y_test = train_test_split(
    list(df["embedding_google"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

X_train_custom, X_test_custom, _, _ = train_test_split(
    list(df["embedding_custom"]), df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

#For ternary :
df_3classes["embedding_google"] = df_3classes["cleaned_review"].apply(lambda x: get_avg_embedding(x, wv_google))
df_3classes["embedding_custom"] = df_3classes["cleaned_review"].apply(lambda x: get_avg_embedding(x, custom_w2v.wv))


# Train-test split (80% training, 20% testing)
X_train_google_ternary, X_test_google_ternary, y_train_ter, y_test_ter = train_test_split(
    list(df_3classes["embedding_google"]), df_3classes["sentiment"], test_size=0.2, random_state=42, stratify=df_3classes["sentiment"]
)

X_train_custom_ternary, X_test_custom_ternary, _, _ = train_test_split(
    list(df_3classes["embedding_custom"]), df_3classes["sentiment"], test_size=0.2, random_state=42, stratify=df_3classes["sentiment"]
)


In [67]:
y_train_binary = np.array(y_train.replace({1: 1, 2: 0}))
y_test_binary = np.array(y_test.replace({1: 1, 2: 0}))

In [84]:
y_train_ternary = np.array(y_train_ter .replace({1: 0, 2: 1,3:2}))
y_test_ternary = np.array(y_test_ter.replace({1: 0, 2: 1,3:2}))

In [68]:
#creating a Feedforward MLP with two hidden layers (50 & 10 nodes).
class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)  # First hidden layer (50 neurons)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(50, 10)  # Second hidden layer (10 neurons)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(10, num_classes)  # Output layer

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x


In [74]:
def train_model(X_train, y_train, X_test, y_test, num_classes, epochs=250, lr=0.001):
    # Convert numpy arrays to PyTorch tensors
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)

    # Define model
    model = MLP(input_size=300, num_classes=num_classes)
    criterion = nn.CrossEntropyLoss()  # Cross-entropy loss
    optimizer = optim.Adam(model.parameters(), lr=lr)  # Adam optimizer

    # Training loop
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

    # Evaluate on test set
    with torch.no_grad():
        test_outputs = model(X_test)
        _, predicted = torch.max(test_outputs, 1)
        accuracy = accuracy_score(y_test.numpy(), predicted.numpy())

    return accuracy


In [75]:
# Training Binary Classification Models:
print("Binary Classification (Positive vs Negative)")

# Train on Google Word2Vec
acc_binary_google = train_model(X_train_google, y_train_binary, X_test_google, y_test_binary, num_classes=2)
print()
print(f"✅ Google W2V Accuracy (Binary): {acc_binary_google:.4f}")
print()
# Train on Custom Word2Vec
acc_binary_custom = train_model(X_train_custom, y_train_binary, X_test_custom, y_test_binary, num_classes=2)
print(f"✅ Custom W2V Accuracy (Binary): {acc_binary_custom:.4f}")




Binary Classification (Positive vs Negative)
Epoch [5/250], Loss: 0.6950
Epoch [10/250], Loss: 0.6876
Epoch [15/250], Loss: 0.6794
Epoch [20/250], Loss: 0.6697
Epoch [25/250], Loss: 0.6571
Epoch [30/250], Loss: 0.6421
Epoch [35/250], Loss: 0.6245
Epoch [40/250], Loss: 0.6048
Epoch [45/250], Loss: 0.5841
Epoch [50/250], Loss: 0.5632
Epoch [55/250], Loss: 0.5431
Epoch [60/250], Loss: 0.5245
Epoch [65/250], Loss: 0.5084
Epoch [70/250], Loss: 0.4947
Epoch [75/250], Loss: 0.4832
Epoch [80/250], Loss: 0.4736
Epoch [85/250], Loss: 0.4657
Epoch [90/250], Loss: 0.4591
Epoch [95/250], Loss: 0.4535
Epoch [100/250], Loss: 0.4488
Epoch [105/250], Loss: 0.4447
Epoch [110/250], Loss: 0.4411
Epoch [115/250], Loss: 0.4380
Epoch [120/250], Loss: 0.4352
Epoch [125/250], Loss: 0.4327
Epoch [130/250], Loss: 0.4305
Epoch [135/250], Loss: 0.4285
Epoch [140/250], Loss: 0.4266
Epoch [145/250], Loss: 0.4248
Epoch [150/250], Loss: 0.4232
Epoch [155/250], Loss: 0.4217
Epoch [160/250], Loss: 0.4202
Epoch [165/250]

In [87]:
print("Ternary Classification (Positive, Negative, Neutral)")
#X_train_google_ternary, X_test_google_ternary
#X_train_custom_ternary, X_test_custom_ternary
# Train on Google Word2Vec
acc_ternary_google = train_model(X_train_google_ternary, y_train_ternary, X_test_google_ternary, y_test_ternary, num_classes=3)
print(f"✅ Google W2V Accuracy (Ternary): {acc_ternary_google:.4f}")

# Train on Custom Word2Vec
acc_ternary_custom = train_model(X_train_custom_ternary, y_train_ternary, X_test_custom_ternary, y_test_ternary, num_classes=3)
print(f"✅ Custom W2V Accuracy (Ternary): {acc_ternary_custom:.4f}")


Ternary Classification (Positive, Negative, Neutral)
Epoch [5/250], Loss: 1.1518
Epoch [10/250], Loss: 1.1393
Epoch [15/250], Loss: 1.1265
Epoch [20/250], Loss: 1.1118
Epoch [25/250], Loss: 1.0951
Epoch [30/250], Loss: 1.0784
Epoch [35/250], Loss: 1.0619
Epoch [40/250], Loss: 1.0474
Epoch [45/250], Loss: 1.0351
Epoch [50/250], Loss: 1.0245
Epoch [55/250], Loss: 1.0146
Epoch [60/250], Loss: 1.0031
Epoch [65/250], Loss: 0.9900
Epoch [70/250], Loss: 0.9761
Epoch [75/250], Loss: 0.9622
Epoch [80/250], Loss: 0.9485
Epoch [85/250], Loss: 0.9354
Epoch [90/250], Loss: 0.9230
Epoch [95/250], Loss: 0.9115
Epoch [100/250], Loss: 0.9010
Epoch [105/250], Loss: 0.8914
Epoch [110/250], Loss: 0.8826
Epoch [115/250], Loss: 0.8746
Epoch [120/250], Loss: 0.8674
Epoch [125/250], Loss: 0.8611
Epoch [130/250], Loss: 0.8553
Epoch [135/250], Loss: 0.8502
Epoch [140/250], Loss: 0.8455
Epoch [145/250], Loss: 0.8411
Epoch [150/250], Loss: 0.8369
Epoch [155/250], Loss: 0.8331
Epoch [160/250], Loss: 0.8294
Epoch [